# Import NOAA La Mauna Observatory atmospheric CO2 data

In [8]:
import urllib.request, urllib.parse, urllib.error
import pandas as pd
import re
import os
import numpy as np

# Custom functions
import Wrangler as Wr
## extractLines
## multipleReplace
## makeDirFile

# Access URL, then request and read temperature data file 
with urllib.request.urlopen('ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt') as fhand:
    # Decode file data and form a list
    file = [line.decode().strip() for line in fhand]

# Identify Index at which CO2 data begins
for c,el in enumerate(file):
    if '#' not in el:
        break

# Extract CO2 data
lst = Wr.extractLines(file[c:])

# Created header list
h_list = re.sub('#+','',file[c-2]).split()
h_list[0:0] = ['Year','Month']

# Convert list to DataFrame
df = pd.DataFrame(lst, columns = h_list) 

# Create Date Index
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
df = df.drop(columns=['decimal'])

# Convert missing data (-99.99) to Null
df['average'] = df.average.replace('-99.99',np.nan)

# Set data to numeric
df[h_list[3:]] = df[h_list[3:]].apply(pd.to_numeric, errors='coerce')

# Export csv file
fullname = Wr.makeDirFile('co2_mm_mlo')
export_csv = df.to_csv(fullname, index = None, header=True)